In [2]:
import torch
import sys
import os

In [1]:
import torch
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from src.pretrained.pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


In [4]:
from pretrained.pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)

In [3]:
print(os.getcwd()) 

c:\Users\rrahman3\PhDDissertation\EUFairGAN\notebooks


In [3]:
sys.path.insert(0, os.path.abspath("../src"))
# import src
from dataloader.celeba_loader import CelebADataLoader


In [27]:
# Load pre-trained model tokenizer (vocabulary)
model = BigGAN.from_pretrained('biggan-deep-128')


INFO:src.pretrained.pytorch_pretrained_biggan.model:loading model biggan-deep-128 from cache at C:\Users\rrahman3\.pytorch_pretrained_biggan\6371c3777477e4e75187da1b9b526561aac3134f38c7299a3438009ae560e20d.3434ebdfa74a8c17e0e56061cfd905fa163c92f110e88df77b47da6ce9910b48
INFO:src.pretrained.pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

c:\Users\rrahman3\PhDDissertation

In [54]:
model.config.num_classes = 14
model.config

{
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 14,
  "output_dim": 128,
  "z_dim": 128
}

In [29]:
model.embeddings = torch.nn.Linear(in_features=14, out_features=128, bias=True)

In [30]:
model

BigGAN(
  (embeddings): Linear(in_features=14, out_features=128, bias=True)
  (generator): Generator(
    (gen_z): Linear(in_features=256, out_features=32768, bias=True)
    (layers): ModuleList(
      (0-2): 3 x GenBlock(
        (bn_0): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=2048, bias=False)
          (offset): Linear(in_features=256, out_features=2048, bias=False)
        )
        (conv_0): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))
        (bn_1): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn_2): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_2): Conv2d(512, 512, kernel_size=(

In [25]:
model.config

{
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

In [20]:
import torch
from torch.nn import Embedding

# Define custom class embedding for medical conditions
num_conditions = 20  # e.g., 20 medical conditions
embedding_dim = model.config.class_embed_dim
model.class_embed = Embedding(num_conditions, embedding_dim)

In [19]:
for name, param in model.named_parameters():
    # if param.requires_grad:
    print(f"Layer: {name:<50} Shape: {param.shape}")

Layer: embeddings.weight                                  Shape: torch.Size([128, 1000])
Layer: generator.gen_z.bias                               Shape: torch.Size([32768])
Layer: generator.gen_z.weight_orig                        Shape: torch.Size([32768, 256])
Layer: generator.layers.0.bn_0.scale.weight_orig          Shape: torch.Size([2048, 256])
Layer: generator.layers.0.bn_0.offset.weight_orig         Shape: torch.Size([2048, 256])
Layer: generator.layers.0.conv_0.bias                     Shape: torch.Size([512])
Layer: generator.layers.0.conv_0.weight_orig              Shape: torch.Size([512, 2048, 1, 1])
Layer: generator.layers.0.bn_1.scale.weight_orig          Shape: torch.Size([512, 256])
Layer: generator.layers.0.bn_1.offset.weight_orig         Shape: torch.Size([512, 256])
Layer: generator.layers.0.conv_1.bias                     Shape: torch.Size([512])
Layer: generator.layers.0.conv_1.weight_orig              Shape: torch.Size([512, 512, 3, 3])
Layer: generator.layers.0.b

In [24]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params

50388609

In [6]:
model

BigGAN(
  (embeddings): Linear(in_features=1000, out_features=128, bias=False)
  (generator): Generator(
    (gen_z): Linear(in_features=256, out_features=32768, bias=True)
    (layers): ModuleList(
      (0-2): 3 x GenBlock(
        (bn_0): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=2048, bias=False)
          (offset): Linear(in_features=256, out_features=2048, bias=False)
        )
        (conv_0): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))
        (bn_1): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn_2): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_2): Conv2d(512, 512, kernel_siz

In [51]:
# Prepare a input
truncation = 0.6
batch_size = 1
# class_vector = one_hot_from_names(['soap bubble', 'coffee', 'mushroom'], batch_size=3)
# class_vector = one_hot_from_names(['bee'], batch_size=batch_size)
class_vector = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=batch_size)

# All in tensors
noise_vector = torch.from_numpy(noise_vector).float()
class_vector = torch.from_numpy(class_vector).float()

# Generate an image
with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)
print(output.shape)

from PIL import Image
from io import BytesIO
import numpy as np
# Assuming output is a PyTorch tensor
# output = output.squeeze(0)  # Remove batch dimension if present
for i in range (0, output.shape[0]):
    print(output[i].shape)
    output_numpy = output[i].permute(1, 2, 0).cpu().numpy()  # Convert from (C, H, W) to (H, W, C)

    # Convert the NumPy array to an image (ensure the values are in [0, 255] for uint8 type)
    # output_image = Image.fromarray((output_numpy * 255).astype('uint8'))
    # obj = obj.transpose((0, 2, 3, 1))
    output_numpy = np.clip(((output_numpy + 1) / 2.0) * 256, 0, 255)

    # Show the image
    print(output_numpy.shape)
    output_numpy = np.asarray(np.uint8(output_numpy), dtype=np.uint8)
    output_image = Image.fromarray(output_numpy, mode="RGB")
    output_image.show()


torch.Size([1, 3, 128, 128])
torch.Size([3, 128, 128])
(128, 128, 3)


In [72]:
# All in tensors
noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)

In [ ]:
# Generate an image
with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)
print(output.shape)

In [ ]:
from PIL import Image
from io import BytesIO
import numpy as np
# Assuming output is a PyTorch tensor
# output = output.squeeze(0)  # Remove batch dimension if present
for i in range (0, output.shape[0]):
    print(output[i].shape)
    output_numpy = output[i].permute(1, 2, 0).cpu().numpy()  # Convert from (C, H, W) to (H, W, C)

    # Convert the NumPy array to an image (ensure the values are in [0, 255] for uint8 type)
    # output_image = Image.fromarray((output_numpy * 255).astype('uint8'))
    # obj = obj.transpose((0, 2, 3, 1))
    output_numpy = np.clip(((output_numpy + 1) / 2.0) * 256, 0, 255)

    # Show the image
    print(output_numpy.shape)
    output_numpy = np.asarray(np.uint8(output_numpy), dtype=np.uint8)
    output_image = Image.fromarray(output_numpy, mode="RGB")
    output_image.show()


In [ ]:
# If you have a GPU, put everything on cuda
noise_vector = noise_vector.to('cuda')
class_vector = class_vector.to('cuda')
model.to('cuda')

In [ ]:
# If you have a GPU put back on CPU
output = output.to('cpu')

In [ ]:
output

In [ ]:
output.shape

In [20]:
output_image.show()

In [ ]:
# Save results as png images
save_as_images(output)